In [1]:
from TSPGraph import Graph
import numpy as np
from numba import jit
from numba import njit, prange
import copy
from tqdm import tqdm
from multiprocessing import Pool
import time

In [19]:
graph = Graph('C')
obj = graph.objective
dist = graph.dist_matrix

In [3]:
@jit
def random_solution():
    sol = np.random.choice(np.arange(200),size=100,replace=False)
    return np.append(sol,sol[0])

C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\3766937091.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit


In [4]:
combination_pairs = [(x,y) for x in range(1,100) for y in range(x+2, 100)]
combination_pairs_edges = [(x,y) for x in range(1,100) for y in range(x+2, 100)]

In [5]:
@jit
def ExchangeVertices(solution, i, j):
    result = copy.deepcopy(solution)
    result[[i,j]] = result[[j,i]]
    return result
@jit
def ExchangeVerticesDelta(dist, solution, i, j):
    res = (dist[solution[i-1], solution[j]] + dist[solution[j], solution[i+1]] + dist[solution[j-1], solution[i]] + dist[solution[i], solution[j+1]]) - (dist[solution[i-1], solution[i]] + dist[solution[i], solution[i+1]] + dist[solution[j-1], solution[j]] + dist[solution[j], solution[j+1]])
    return res
@jit
def MinInAllExchangedVertices(dist, solution):
    minDelta = 0
    resultExchange = tuple()
    for x in combination_pairs:
        delta = ExchangeVerticesDelta(dist, solution, x[0], x[1])
        if delta < minDelta:
            minDelta = delta
            resultExchange = (x[0], x[1])
    return [minDelta, resultExchange]
@jit
def GreedyExchangeVertex(dist, solution):
    for x in combination_pairs:
        delta = ExchangeVerticesDelta(dist, solution, x[0], x[1])
        if delta < 0:
            return (x[0], x[1])
    return None


MinInAllExchangedVertices(dist, random_solution())

C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\2949073171.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\2949073171.py:6: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\2949073171.py:10: NumbaDeprecationWarning: The 'nopython' keyword argument was no

[-9835, (19, 31)]

In [6]:
@jit
def ExchangeEdges(solution, i, j):
    assert i<j
    result = copy.deepcopy(solution)
    result = np.concatenate((result[:i], result[j:i-1:-1], result[j+1:]))
    return result
@jit
def ExchangeEdgesDelta(dist, solution, i, j):
    assert i<j
    res = (dist[solution[i-1], solution[j]] + dist[solution[i], solution[j+1]]) - (dist[solution[i-1], solution[i]] + dist[solution[j], solution[j+1]])
    return res
@jit
def MinInAllExchangedEdges(dist, solution):
    minDelta = 0
    resultExchange = tuple()
    for x in combination_pairs_edges:
        delta = ExchangeEdgesDelta(dist, solution, x[0], x[1])
        if delta < minDelta:
            minDelta = delta
            resultExchange = (x[0], x[1])
    return (minDelta, resultExchange)
@jit
def GreedyExchangeEdges(dist, solution):
    for x in combination_pairs_edges:
        delta = ExchangeEdgesDelta(dist, solution, x[0], x[1])
        if delta < 0:
            return (x[0], x[1])
    return None

MinInAllExchangedEdges(dist, random_solution())

C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\432162731.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\432162731.py:7: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\432162731.py:12: NumbaDeprecationWarning: The 'nopython' keyword argument was not s

(-6949, (19, 59))

In [7]:
@jit
def SwapVertexWithNewOne(solution, i, a):
    result = copy.deepcopy(solution)
    result[i] = a
    return result
@jit
def SwapVertexWithNewOneDelta(obj, solution, i, a):
    return (obj[solution[i-1], a] + obj[a, solution[i+1]]) - (obj[solution[i-1], solution[i]] + obj[solution[i], solution[i+1]])
@jit
def MinInAllSwaps(solution, obj):
    domain = np.array(range(200))
    not_in_sol = domain[np.isin(domain, solution, invert=True)]
    #pairs of index in current cycle - node 
    pairs = np.transpose([np.tile(domain[:100], 100), np.repeat(not_in_sol, 100)])
    minDelta = 0
    resultSwap = tuple()
    for x in pairs:
        delta = SwapVertexWithNewOneDelta(obj, solution, x[0], x[1])
        if delta < minDelta:
            minDelta = delta
            resultSwap = (x[0], x[1])
    return (minDelta, resultSwap)
@jit
def GreedySwap(solution, obj):
    domain = np.array(range(200))
    not_in_sol = domain[np.isin(domain, solution, invert=True)]
    #pairs of index in current cycle - node 
    pairs = np.transpose([np.tile(domain[:100], 100), np.repeat(not_in_sol, 100)])
    for x in pairs:
        delta = SwapVertexWithNewOneDelta(obj, solution, x[0], x[1])
        if delta < 0:
            return (x[0], x[1])
    return None
print(MinInAllSwaps(random_solution(), obj))

C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\3288179290.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\3288179290.py:6: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\3288179290.py:9: NumbaDeprecationWarning: The 'nopython' keyword argument was not

(-6340, (68, 196))


In [8]:
@jit
def SteepestLocalSearch(obj, dist, initial_solution, type):
    current_solution = initial_solution
    max_iter = 1_000_000
    while(max_iter > 0):
        # print(current_cost)
        best_swap_val, best_swap = MinInAllSwaps(current_solution, obj)
        if type == 'edge':
            best_exchange_val, best_exchange = MinInAllExchangedEdges(dist, current_solution)
        else:
            best_exchange_val, best_exchange = MinInAllExchangedVertices(dist, current_solution)
        
        if best_swap_val >= 0 and best_exchange_val >= 0:
            break

        if best_swap_val < best_exchange_val:
            current_solution = SwapVertexWithNewOne(current_solution, best_swap[0], best_swap[1])
            # print('swap' + str(best_swap_val))
        elif type == 'edge':
            current_solution = ExchangeEdges(current_solution, best_exchange[0], best_exchange[1])
            # print('exchange ' + str(best_exchange_val) + str(best_exchange))
        else:
            current_solution = ExchangeVertices(current_solution, best_exchange[0], best_exchange[1])
            # print('exchange ' + str(best_exchange_val) + str(best_exchange))

        max_iter -= 1
    return current_solution
        
SteepestLocalSearch(obj, dist, random_solution(), 'edge')

C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\2515397583.py:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit
C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\2515397583.py:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "SteepestLocalSearch" failed type inference due to: Invalid use of type(CPUDispatcher(<function MinInAllSwaps at 0x000001CF390D3060>)) with parameters (array(int64, 1d, C), array(int64, 2d, C))

During: resolving callee type: type(CPUDispatcher(<function MinInAllSwaps at 0x000001CF390D3060>))
During: typing of call at C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\2515397583.py (7)


File "C:\U

array([ 29,  33,  73, 136,  18,  52, 139, 107, 109,  12, 132, 185, 170,
       100, 181,  87, 187, 147, 159,  89,  64, 129, 171,  58,  72,  85,
       114,  71, 193,  44, 117, 150, 162, 158,  67,  78, 156,  51,  91,
        70, 174, 140, 148, 141, 130, 142,  69, 113,  82,  63,   8,  16,
       172,  40, 163,   2, 182, 167,   5, 144, 178, 128,  34, 183, 197,
        26,  92, 122, 143, 179,  31,   1, 101,  24, 127,  50,  49, 102,
        37, 165, 154,  25,  36, 137,  56,  88, 153,  55,  80, 157,  57,
        99,   0,  66, 169, 198, 135,  19, 190,  95,  22], dtype=int64)

In [9]:
def GreedyLocalSearch(obj, dist, initial_solution, type):
    current_solution = initial_solution
    max_iter = 1_000_000
    while(max_iter > 0):
        # print(current_cost)
        max_iter -= 1

        swap = GreedySwap(current_solution, obj)
        if swap is not None:
            current_solution = SwapVertexWithNewOne(current_solution, swap[0], swap[1])
            continue
        if type == 'edge':
            exchange = GreedyExchangeEdges(dist, current_solution)
            if exchange is not None:
                current_solution = ExchangeEdges(current_solution, exchange[0], exchange[1])
                continue
        else:
            exchange = GreedyExchangeVertex(dist, current_solution)
            if exchange is not None:
                current_solution = ExchangeVertices(current_solution, exchange[0], exchange[1])
                continue
        break
    return current_solution
        
GreedyLocalSearch(obj, dist, random_solution(), 'edge')

C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\3288179290.py:23: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "GreedySwap" failed type inference due to: No implementation of function Function(<built-in function array>) found for signature:
 
 >>> array(range_state_int64)
 
There are 2 candidate implementations:
    - Of which 2 did not match due to:
    Overload in function 'impl_np_array': File: numba\np\arrayobj.py: Line 5384.
      With argument(s): '(range_state_int64)':
     Rejected as the implementation raised a specific error:
       TypingError: The argument "object" must be array-like
  raised from C:\Users\cichy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numba\np\arrayobj.py:5388

During: resolving callee type: Function(<built-in function array>)
During: typing of call at C:\Users\cichy\AppData\Local\Temp\ipykernel_8672\3288179290.py (25)


array([ 31, 101,   1,  38, 103, 125,  50,  49, 116, 102,  23, 154, 123,
        25,   4,  56,  80, 145,  86, 120,  19,   6,  33, 168,  29,  61,
        10,  90, 100, 170,  12,  97, 139,  59, 193,  71,  28, 104,  87,
        35,  58,  89, 171,  72,  85, 110, 126,  44, 196,  60,  21,   9,
         3,  91,  70, 174, 148, 130,  13, 149,  32,  30,  20,  63,   8,
         7,  84,  14,  65, 132,  18, 106,  16,  22, 172,  74, 118,  54,
       133, 182,  17,   2, 167,   5,  11,  34, 183, 127,  24,  26, 128,
       135, 175,  57,  15, 164,   0, 169,  66,  27, 111], dtype=int64)

In [10]:
def cycle_cost(obj, cycle):
        cycle_segments = (cycle[:-1], cycle[1:])
        return np.sum(obj[cycle_segments])

In [11]:
def MSLS(obj, dist, iterations=200):
    best_result = 999999999999
    for i in range(iterations):
        result = cycle_cost(obj, SteepestLocalSearch(obj, dist, random_solution(), 'edge'))
        if result < best_result:
            best_result = result
    return best_result
MSLS(obj, dist, iterations=20)

47604

In [12]:
from multiprocessing.dummy import Pool as ThreadPool
from itertools import permutations

def msls_experiment(iter=20):
    return MSLS(obj,dist,iter)
    
l = [200 for _ in range(20)]

with ThreadPool(10) as p:
    data = p.map(msls_experiment, l)
    print(data)


[47325, 46701, 46997, 46850, 47136, 46294, 46921, 46857, 47195, 46678, 46644, 46405, 46146, 46925, 46404, 46825, 46610, 46628, 46832, 46767]


In [13]:
print(np.min(data), np.max(data), np.mean(data), np.median(data))

46146 47325 46757.0 46796.0


In [14]:
def perturbation(solution):
    domain = np.array(range(200))
    not_in_sol = domain[np.isin(domain, solution, invert=True)]
    return SwapVertexWithNewOne(solution, np.random.randint(1,100), np.random.choice(not_in_sol,1))

def n_perturbations(solution, n):
    for i in range(n):
        solution = perturbation(solution)
    return solution

In [22]:
def ILS(obj, dist, initial_solution, search_time, n):
    curr_solution = SteepestLocalSearch(obj, dist, initial_solution, 'edge')
    best_solution = cycle_cost(obj, curr_solution)
    timeout = time.time() + search_time
    ls_counter = 1
    while(time.time() < timeout):
        curr_solution = n_perturbations(curr_solution, n)
        curr_solution = SteepestLocalSearch(obj, dist, curr_solution, 'edge')
        ls_counter+=1
        new_cost = cycle_cost(obj, curr_solution)
        if new_cost < best_solution:
            best_solution = new_cost
    return (best_solution, ls_counter)

ils_results = []   
ils_counters = []
initial_solution = random_solution()
for j in range(20):
    scr, counter = ILS(obj, dist,initial_solution, search_time=400, n=5)
    ils_results.append(scr)
    ils_counters.append(counter)
ils_results, ils_counters

C:\Users\cichy\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\numpy\core\fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)


KeyboardInterrupt: 

In [ ]:
print(np.min(ils_results), np.max(ils_results), np.mean(ils_results), np.median(ils_results))
print(np.min(ils_counters), np.max(ils_counters), np.mean(ils_counters), np.median(ils_counters))

45389 61282 48800.2 46427.5
